# Project scope

Create a classifier that predicts how my day was based on inputs.

Simplest possible pipeline just uses the productive and unproductive hours. 

Assumptions: the survey methodology was consistent throughout (which it mostly was). So I'm looking to predict how my day will be based on what I do during the day. It doesn't predict what I do during my day (I guess we could do that too, but it would be wildly inaccurate without a pretty insane number of features). 

The hours are a number from one through 8 (I was inconsistent with that, sometimes I would write higher numbers but really it should cap at 8). So there's a preprocessing step there.

Note this is a pretty small amount of data.

In [1]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Get data

The data is from Sheets - I can even do my cleaning there if I want. That's probably better for the actual correction of individual pieces of data. But if there is actual preprocessing that needs to be done, it should be done in Python.

The preprocessing step is just dropping NaNs - so if we had a month worth of new test data coming in in the same format, we would just drop any NaNs before running. 

In [2]:
path = "base data (dnc)/Daily Recap (Responses) - RAW form responses.csv"
target = 'How would you rate your overall satisfaction for the day?'

df_full = pd.read_csv(path).dropna(subset=[target])  # ensure no missing y values

train, test, y, y_test = train_test_split(df_full.drop(target, axis=1), df_full[target], random_state = 0)
# train.columns
# df_full.isnull().sum()

Note that right now, I'm limiting all of the features up front. Later when I do feature selection, I'd probably limit the features here. Is there harm in doing that now instead?

In [3]:
features = ['Productive AI Hours', 'Productive Hours', 'Unproductive Hours']
X = train[features].copy()
X_test = test[features].copy()

In [4]:
X.isnull().sum()

Productive AI Hours    0
Productive Hours       0
Unproductive Hours     0
dtype: int64

I have an extra step to make sure X and y have the same number of features... that's because y doesn't usually have any NaNs. 

(!) duh, i'm struggling to find ways to drop the same X and y row because you usually don't drop rows. either drop the entire column, or you impute the value. The quick fix is for me to just add in the data though. 

I guess I could technically just test it on this test set for starters. That's pretty bad practice, but it should give me a rough idea of how predictive this is. 

In [5]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50, random_state=0)  # very basic model (?) plot change in performance vs training time?

In [6]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

print("Accuracy scores (in %):\n", scores*100)
print("Average accuracy score (across experiments):")
print(scores.mean()*100)

Accuracy scores (in %):
 [45.83333333 52.08333333 47.91666667 33.33333333 39.58333333]
Average accuracy score (across experiments):
43.75


Wow, that's kind of low. But not that surprising give that I just gave it a bunch of numbers. Wait, does y need to be encoded?

I guess let's get the scoring working and then we can see how much it improves it. 

In [7]:
model.fit(X, y)
model.score(X_test, y_test)

0.45

I think I just want to be better than picking the same guess every time. Random would be 20% expected accuracy, I need to do exploratory analysis to determine what the most common value is (e.g. if I just guessed "okay day" every day). I hope it's under 50%. Oh shit, is this data imbalanced?